Using the [QMENTA SDK](https://www.qmenta.com/sdk/) you can easily package and deploy your predictive model to the QMENTA Platform in order to run it in a scalable cloud environment.
You can learn more about the SDK from a technical perspective at https://docs.qmenta.com/sdk/.

If you registered to the QMENTA Platform using the **promotional code** shown in [1.qmenta_client_download_data.ipynb](./1.qmenta_client_download_data.ipynb), you already have **developer access** to the platform. 

With this level of access you can create, edit and delete **analysis tools** in the platform. You will first neeed to package your applications into a Docker container image, and then upload it to [Docker Hub](https://hub.docker.com/) so that the platform can fetch it and securely run it.

![My Tools view](./assets/my_tools.png)

In this Jupyter notebook we will show you how to:
- Write an script that does the prediction and is integrated with the QMENTA Platform via the QMENTA SDK
- Build a Docker image that packages everything needed to perform inference
- Upload the produced Docker image to Docker Hub
- Register our tool in **My Tools**
- Run the tool on one subject of the project

# 1. Prepare the SDK script

Our script will basically load the persisted model from disk, load the data to be predicted from disk and run inference on this model using the loaded data. On top of that, the script will communicate with the platform in order to download input data and resources, set the progress of the analysis and finally upload the results back to the platform, so that they can be accessed via the web interface.

An example of such script can be found in [predict_age.py](./predict_age.py)

In order to understand the mechanics of SDK tool development please check this [documentation page](https://docs.qmenta.com/sdk/using_sdk.html).

# 2. Build the Docker Image

### Install Docker

Head over to the [Get Docker](https://docs.docker.com/install/#desktop) documentation page and follow the **Docker Community Edition** installation process for your platform of choice.

### Use a Dockerfile

A Dockerfile allows you to programatically specify how you build the Docker images without having to manually modify a container and create a snapshot of it.

You can check the example [Dockerfile](./Dockerfile) included in this repository. For more details about the Dockefile syntax check the documentation [here](https://docs.docker.com/engine/reference/builder/).

### Build the image

We will use the `docker build` command to build our Docker image, that later we will push to the default public Docker registry (DockerHub)

First open a terminal and type the following commands:
```
user@hostname : ~ $ cd ~/qmenta_cnic_1000_brains_challenge/
user@hostname : ~ $ mkdir docker_build
```

In this `docker_build` folder we will put all the necessary files to package our application as a Docker image. Assuming that you have cloned this repository in `~/1000_brains_challenge`:
```
user@hostname : ~ $ cp ~/1000_brains_challenge/Dockerfile ./docker_build/Dockerfile
user@hostname : ~ $ cp ~/1000_brains_challenge/predict_age.py ./docker_build/predict_age.py
user@hostname : ~ $ cp ~/1000_brains_challenge/environment.yml ./docker_build/environment.yml
user@hostname : ~ $ cp ~/qmenta_cnic_1000_brains_challenge/models/linear_regression_example.pkl ./docker_build/linear_regression_example.pkl
```

Finally we will run the docker build command:
```
user@hostname : ~ $ docker build -t 1000_brains_example ./docker_build
```

# 3. Upload your Docker image to DockerHub

First you will need to [create an account in DockerHub](https://hub.docker.com/). By creating an account you will have the possibility to upload all your images to DockerHub and make them available to anybody. The [Free Account plan](https://hub.docker.com/billing-plans/) also includes one private image, which can only be accessed with your credentials.

TODO: Continue here

# 4. Register your tool

# 5. Run the tool in the platform